In [7]:
from pyspark.sql import functions as F

d0 = spark.table("dbo.bronze_openaq_sensor_days_v2")
d = d0.drop("location_id")
s = spark.table("dbo.bronze_openaq_location_sensors_nyc_v2").select(F.col("id").alias("sensor_id"), F.col("location_id").alias("loc_id"))
l = spark.table("dbo.bronze_openaq_locations_nyc_v2").select(F.col("id").alias("loc_id"), F.col("name").alias("location_name"), F.col("timezone").alias("timezone"), F.col("`coordinates.latitude`").alias("lat"), F.col("`coordinates.longitude`").alias("lon"))
opaq = d.join(s, on="sensor_id", how="inner").join(l, on="loc_id", how="inner").withColumn("day_utc", F.to_date("period_datetime_from_utc")).groupBy("day_utc", "loc_id", "location_name", "timezone", "lat", "lon", "parameter_name") \
    .agg(F.avg(F.col("value").cast("double")).alias("avg_value"), F.min(F.col("value").cast("double")).alias("min_value"), F.max(F.col("value").cast("double")).alias("max_value"), F.count("*").alias("obs_cnt")).withColumnRenamed("loc_id", "location_id")

opaq.write.format("delta").mode("overwrite").option("overwriteSchema","true").saveAsTable("dbo.silver_openaq_daily_nyc_v2")
print("opaq rows:", spark.table("dbo.silver_openaq_daily_nyc_v2").count())
spark.table("dbo.silver_openaq_daily_nyc_v2").groupBy("parameter_name").count().show()

StatementMeta(, 57313b1e-4291-4fb7-9057-9be2bbc56e01, 9, Finished, Available, Finished)

opaq rows: 490
+--------------+-----+
|parameter_name|count|
+--------------+-----+
|           no2|  160|
|          pm25|  330|
+--------------+-----+

